## Gaussian Discriminant Analysis

In [31]:
import pandas as pd
import numpy as np

# To reset the printoptions
# np.set_printoptions(edgeitems=3,infstr='inf',linewidth=75, nanstr='nan', precision=8,suppress=False, threshold=1000, formatter=None)

np.set_printoptions(precision=4, suppress=True)

#np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

### For multi-class label y ={0, 1, 2, ...}

In [32]:
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
y = dataset.target
#y=(y>0).astype(int) 

target_names = list(dataset.target_names)
print(target_names)

['setosa', 'versicolor', 'virginica']


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [34]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112, 4), (38, 4), (112,), (38,))

In [70]:
class GaussianDiscAnalysis: 
    def compute_phi(self, y):
        n = len(y)
        phi = dict()
        for idx in range(len(self.classes)):
            phi[idx] = (1/n) * np.sum(y==idx)
        return phi
    
    def compute_mu(self, X, y):
        mu_dict = dict()
        for idx in range(len(self.classes)):
            # Add mu for each class
            mu_dict[idx] = np.sum(X[y==idx], axis=0)/ np.sum(y==idx)
        return mu_dict

    def compute_sigma(self, X, y):
        n = len(X)
        #y = y.reshape(-1,1)
        Xmu = X.copy()
        for idx in range(len(self.classes)):
            Xmu = Xmu \
              - self.mu[idx]*np.ones_like(Xmu)*(y==idx).reshape(-1,1)
        return (1/n) * Xmu.T@Xmu
    
    
    def compute_Pxyi(self, X, idx):
        """Probability of X given y"""
        m = X.shape[1]
        sigma_inv = np.linalg.inv(self.sigma)
        det_sigma = np.linalg.det(self.sigma)
        #mu_i = mu(X, y, idx)
        Pxi = (1/((2*np.pi)**(m/2))) \
                *(1/(det_sigma**0.5)) \
                * np.exp(- 0.5*np.sum(((X-self.mu[idx])@sigma_inv)*(X-self.mu[idx]), axis=1))
    #     Pxi = np.log(1) \
    #             - np.log((2*np.pi)**(m/2)) \
    #             - np.log(np.sqrt(det_sigma)) \
    #             - np.sum(((X-mu_i)@sigma_inv)*(X-mu_i), axis=1)
        return Pxi
    
    def fit(self, X, y):
        """Computes mean, covariance and proabilities of y (phi)"""
        self.classes = np.unique(y)
        self.mu = self.compute_mu(X, y)
        self.sigma = self.compute_sigma(X, y)
        self.phi = self.compute_phi(y)
        
    def predict_proba(self, X):
        """Computes the probability of example belonging to that class"""
        n = len(X)
        Pyi = np.zeros((n, len(self.classes)))
        
        for idx in range(len(self.classes)):
            #print(self.compute_Pxyi(X, idx))
            py_i = self.compute_Pxyi(X, idx) * self.phi[idx]
            Pyi[:, idx] = py_i
        return Pyi
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)
    
    def generate_data(self, class_id, num_samples=1):
        """Generates new unseen dataset from a normal distribution
            given the mean of class and covariance
        """
        mean = self.mu[class_id]
        cov = self.sigma
        return np.random.multivariate_normal(mean, cov, num_samples)
        

In [71]:
GDA = GaussianDiscAnalysis()

In [72]:
GDA.fit(X_train, y_train)

In [84]:
predictions = GDA.predict(X_test)

In [85]:
predictions

array([0, 1, 0, 0, 2, 1, 2, 1, 2, 0, 0, 1, 0, 2, 1, 2, 2, 0, 1, 2, 1, 1,
       2, 2, 2, 1, 1, 0, 0, 1, 0, 1, 2, 2, 0, 0, 0, 2])

In [86]:
GDA.predict_proba(X_test)[0:5]

array([[0.1112, 0.    , 0.    ],
       [0.    , 0.2429, 0.    ],
       [0.3827, 0.    , 0.    ],
       [0.5792, 0.    , 0.    ],
       [0.    , 0.0001, 0.7823]])

In [87]:
np.sum(predictions == y_test) / len(y_test)

1.0

### Generate new data

In [107]:
new_data = GDA.generate_data(class_id=1, num_samples=20)

In [108]:
new_data

array([[5.9878, 3.1591, 4.3618, 1.63  ],
       [6.2657, 3.203 , 4.5718, 1.5006],
       [5.3629, 2.5017, 4.081 , 1.145 ],
       [6.8119, 2.7217, 4.8029, 1.036 ],
       [6.0458, 3.0751, 3.923 , 1.2568],
       [6.0077, 2.9935, 3.8469, 1.1896],
       [7.1471, 3.403 , 5.0233, 1.7937],
       [5.561 , 2.2436, 3.8857, 1.09  ],
       [6.2727, 2.3104, 4.8451, 1.1651],
       [6.1436, 3.3423, 4.2712, 1.4712],
       [5.929 , 2.9156, 4.4914, 1.6332],
       [5.6817, 2.3131, 3.9704, 0.9838],
       [6.0336, 2.9637, 4.9055, 1.5721],
       [6.3244, 3.366 , 4.617 , 1.5185],
       [5.5489, 1.9019, 4.3654, 1.1868],
       [5.9388, 2.9445, 3.9573, 1.1754],
       [5.7966, 2.5646, 4.2907, 0.9156],
       [6.6072, 2.8884, 4.6633, 1.4183],
       [5.7243, 2.8922, 3.4603, 0.8527],
       [5.626 , 2.093 , 3.8071, 1.1259]])

In [106]:
GDA.predict(new_data)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])